In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.listdir()

['.ipynb_checkpoints', 'Corona.ipynb']

In [3]:
os.chdir('data')

In [4]:
os.listdir()

['test', 'train']

In [5]:
os.chdir('..')

In [6]:
os.listdir()

['.ipynb_checkpoints', 'Corona.ipynb', 'data']

In [2]:
train_path = 'data/train'
test_path = 'data/test'
valid_path = 'data/valid'

In [9]:
print(os.listdir(train_path))
print(os.listdir(test_path))

['covid', 'non']
['covid', 'non']


In [18]:
len(os.listdir(test_path+'/covid'))

35

In [21]:
len(os.listdir(valid_path+'/covid'))

15

In [25]:
len(os.listdir(train_path+'/non'))

1436

In [10]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [31]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), batch_size=10)

Found 1570 images belonging to 2 classes.


In [32]:
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path, target_size=(224,224), batch_size=10)

Found 479 images belonging to 2 classes.


In [3]:
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 135 images belonging to 2 classes.


In [34]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32

In [35]:
mobile.count_params()

4253864

In [36]:
x = mobile.layers[-6].output
output = Dense(units=2, activation='sigmoid')(x)

In [37]:
model = Model(inputs=mobile.input, outputs=output)

In [38]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [39]:
for layer in model.layers[:-23]:
    layer.trainable = False

In [40]:
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [41]:
model.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=valid_batches, 
          validation_steps=len(valid_batches), epochs=15, verbose=2)

Epoch 1/15
157/157 - 145s - loss: 0.1544 - accuracy: 0.9688 - val_loss: 0.2326 - val_accuracy: 0.9207
Epoch 2/15
157/157 - 149s - loss: 0.0217 - accuracy: 0.9994 - val_loss: 0.0457 - val_accuracy: 0.9916
Epoch 3/15
157/157 - 141s - loss: 0.0123 - accuracy: 0.9987 - val_loss: 0.0357 - val_accuracy: 0.9916
Epoch 4/15
157/157 - 161s - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.0323 - val_accuracy: 0.9896
Epoch 5/15
157/157 - 159s - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0276 - val_accuracy: 0.9896
Epoch 6/15
157/157 - 149s - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0290 - val_accuracy: 0.9979
Epoch 7/15
157/157 - 161s - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0222 - val_accuracy: 0.9937
Epoch 8/15
157/157 - 143s - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0211 - val_accuracy: 0.9916
Epoch 9/15
157/157 - 110s - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0195 - val_accuracy: 0.9937
Epoch 10/15
157/157 - 107s - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0223 - 

In [6]:
test_labels = test_batches.classes

In [7]:
test_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1], dtype=int32)

In [45]:
predictions = model.predict(x=test_batches, verbose=2)

14/14 - 9s


In [49]:
pred = predictions.argmax(axis=1)

In [50]:
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1])

In [51]:
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))

In [52]:
cm

array([[ 15,   0],
       [  0, 120]])

In [1]:
from tensorflow.keras.models import model_from_json

In [54]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [5]:
# To load the model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
predictions = loaded_model.predict(x=test_batches, verbose=2)

In [2]:
def prepare_img(file):
    img_path = ''
    img = image.load_img(img_path+file, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array_extended_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_extended_dims)